In [2]:
# README

# Unlike other optimization problems, we don't optimize the weights. Here, we try to find the pixels of the 
# final(content+style) image by reducing the error of the cost function.

# There are two cost functions:
# 1. J_content
# 2. J_style

# Total cost function: 
# J_total = alpha*J_content + beta*J_style

# J_content:
# 

In [ ]:
import numpy as np
import tensorflow as tf